In [ ]:
# pc
from collections import defaultdict
import numpy as np

In [5]:
# pc
train = np.loadtxt('sample_data/movielens_100k_u1.base', skiprows=0, delimiter='\t')
test = np.loadtxt('sample_data/movielens_100k_u1.test', skiprows=0, delimiter='\t')

In [6]:
train = train[:, :-1]
test = test[:, :-1]

In [9]:
train = train.astype('int')
test = test.astype('int')

In [11]:
ratingsPerUser = defaultdict(list)
ratingsPerMovie = defaultdict(list)
for u, m, r in train:
  ratingsPerUser[u].append((m, r))
  ratingsPerMovie[m].append((u, r))

In [14]:
global_rating = train[:, -1].mean()

In [21]:
def rmse(predict, true):
    return np.sqrt(sum((true[i] - predict[i]) ** 2 for i in range(len(true)))\
                   / len(true))


In [13]:
def Jaccard(s1, s2):
    num = s1.intersection(s2)
    denom = s1.union(s2)
    if denom != 0: return len(num) /len(denom)
    else: return 0

In [15]:
def predict_rating(u, m):
    if u in ratingsPerUser and m in ratingsPerMovie:
        users_watched_m, _ = zip(*ratingsPerMovie[m])
        u_movies, _ = zip(*ratingsPerUser[u])
        sims = []
        for other_u in users_watched_m:
            if other_u == u: continue
            other_u_movies, _ = zip(*ratingsPerUser[other_u])
            sims.append((Jaccard(set(u_movies), set(other_u_movies)), other_u))
        if sims:
            sim_user = max(sims)[-1]
            for sim_movie, r in ratingsPerUser[sim_user]:
                if sim_movie == m: return r
        else: return global_rating
    else: return global_rating

In [16]:
predict = [predict_rating(u, m) for u, m in train[:, :-1]]
true = train[:, -1]

In [22]:
train_rmse = rmse(predict, true)

In [18]:
predict_test = [predict_rating(u, m) for u, m in test[:, :-1]]
true_test = test[:, -1]

In [23]:
test_rmse = rmse(predict_test, true_test)

In [24]:
train_rmse, test_rmse

(1.3454806932601842, 1.359711250948524)